In [3]:
import random
import pandas as pd
import numpy as np
# Example: Load your dataset
courses_df = pd.read_csv('MOOC.csv')  # change the path if needed

# Initialize Population
def initialize_population(pop_size, course_pool, chromosome_length):
    return [random.sample(course_pool, chromosome_length) for _ in range(pop_size)]

# Fitness function with weighted preferences
def fitness(chromosome, user_preferences, courses_df, weights):
    score = 0
    for course in chromosome:
        course_data = courses_df[courses_df['Course Name'] == course].iloc[0]

        # Rating contribution
        try:
            rating = float(course_data['Course Rating'])
        except:
            rating = 0.0  # if 'Not Calibrated' or invalid
        score += weights['rating'] * rating

        # Skill match contribution
        all_skills = course_data['all_skill']
        if isinstance(all_skills, str):
            if any(skill.lower() in all_skills.lower() for skill in user_preferences['desired_skills']):
                score += weights['skill_match'] * 1  # skill matched

        # Difficulty level match contribution
        if isinstance(course_data['Difficulty Level'], str):
            if course_data['Difficulty Level'].lower() == user_preferences['difficulty'].lower():
                score += weights['difficulty_match'] * 1  # difficulty matched

    return score

# Selection: roulette wheel based
def selection(population, fitnesses):
    selected = random.choices(population, weights=fitnesses, k=2)
    return selected

# Crossover
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 2)
    child = parent1[:point] + parent2[point:]
    return child

# Mutation
def mutate(chromosome, course_pool):
    index = random.randint(0, len(chromosome) - 1)
    new_gene = random.choice(course_pool)
    while new_gene in chromosome:
        new_gene = random.choice(course_pool)
    chromosome[index] = new_gene
    return chromosome

# Main Genetic Algorithm function


def genetic_algorithm(user_preferences, courses_df, generations=100, pop_size=20, chromosome_length=5, weights=None):
    course_pool = courses_df['Course Name'].tolist()
    population = initialize_population(pop_size, course_pool, chromosome_length)

    for gen in range(generations):
        fitnesses = [fitness(chromosome, user_preferences, courses_df, weights) for chromosome in population]
        
        # Handle invalid fitnesses
        fitnesses = [0 if (f is None or np.isnan(f) or np.isinf(f)) else f for f in fitnesses]
        
        if sum(fitnesses) == 0:
            # If all fitness scores are 0, just randomly select parents
            new_population = [random.choice(population) for _ in range(pop_size)]
        else:
            new_population = []
            for _ in range(pop_size // 2):
                parent1, parent2 = selection(population, fitnesses)
                child1 = crossover(parent1, parent2)
                child2 = crossover(parent2, parent1)
                child1 = mutate(child1, course_pool)
                child2 = mutate(child2, course_pool)
                new_population.extend([child1, child2])

        population = new_population

    # After evolution, return the best chromosome
    fitnesses = [fitness(chromosome, user_preferences, courses_df, weights) for chromosome in population]
    best_index = fitnesses.index(max(fitnesses))
    return population[best_index]


# Example Usage
if __name__ == "__main__":
    user_preferences = {
        'desired_skills': ['Machine Learning', 'Data Science'],  # can give multiple skills
        'difficulty': 'Intermediate'
    }

    weights = {
        'rating': 0.1,           # 10% importance to ratings
        'skill_match': 0.8,      # 80% importance to skill match
        'difficulty_match': 0.1  # 10% importance to difficulty match
    }

    best_courses = genetic_algorithm(
        user_preferences,
        courses_df,
        generations=100,
        pop_size=30,
        chromosome_length=5,
        weights=weights
    )

    print("\nRecommended Courses:")
    for course in best_courses:
        print("-", course)



Recommended Courses:
- Academic Discussions in English
- TensorFlow for AI: Get to Know Tensorflow
- Building a Hiring Plan by Analyzing Past Data in Sheets
- Machine Learning With Big Data
- Career Planning:  A Pathway to Employment
